# Analyze and Visualize Results

In [2]:
import logging
import json
from pathlib import Path
from typing import List, Tuple, Union, Dict, Optional
import sys
from multiprocessing import Process, Pool
import psutil
import base64
import asyncio
import cv2
import imageio as iio
import numpy as np
from IPython.core.display import HTML
from matplotlib import pyplot as plt
from scipy.fft import fft, fftfreq, fftn, fftshift

from turbx import REPO_PATH, log
from turbx.vis import (calc_tfpnr, calc_tdr, calc_frames_removed, label_to_per_frame_list)
from turbx.metrics import (
    calc_box_area,
    boxes_to_binary,
    target_detection_rate,
    tfpnr,
)

[DEBUG] 11/10/2022 12:44:44PM: MainProcess: __init__.py - Loaded logging config file: /home/bl33m/turbx/src/turbx/logging.yaml


### Load experiment output

In [3]:
base_path = f"{REPO_PATH}/notebooks/outputs/"
#exp_path = "2022-10-27/10-57-06/0"  # no tracklet - best
#exp_path = "2022-10-27/13-06-28/0"  # tracklet low fpr - best FPR
#exp_path = "2022-10-27/13-35-41/0"   # tracklet low fnr
#exp_path = "2022-10-28/13-48-14/0"  # tracklet w/ params as no tracklet
exp_path = "/0" # optimized
exp_path = Path(base_path + exp_path)

all_results = []
for f_name in Path(exp_path).glob("**/*.json"):
    with open(f_name, 'r') as f:
        params = json.load(f)
        all_results.append(params)
        

### Calculate metrics on experiment data

In [33]:
import os

# calculate per video scores

# Things we want per run,
# 1) mAR
# 2) mAP
# 3) F1
# 4) TDR - if this is not 1 then we can adjust this param search
# TDR can be tought of as the notification rate, we really do not want to miss a notification

all_neg_dets= 0
all_neg_lab= 0
all_frames =0


maxF1 = 0
maxAP = 0
maxAR = 0
maxFramesRemoved = 0
APs = []
ARs = []
F1s = []
FRs = []

#print("Per frame:")
#print("id:, FPR, FNR, Target Detection Rate, % Frames Removed, % Neg Frames Removed")

base_path = f"{REPO_PATH}/notebooks/outputs/"

for i, param_set in enumerate(os.listdir(base_path)):
    param_set_path = Path(base_path + param_set)
    param_set_results = []
    for f_name in Path(param_set_path).glob("**/*.json"):
        with open(f_name, 'r') as f:
            params = json.load(f)
            param_set_results.append(params)
    tp = 0
    fp = 0
    fn = 0
    all_neg_dets = 0
    all_neg_lab = 0
    all_frames = 0

    # this is for each video
    for result in param_set_results:
        binary_label, binary_pred, tfpnr_dict = calc_tfpnr(result['label'], result['prediction'], show=False, save=False)
        unique_targs, det_targs, tdr = calc_tdr(result['label'], result['prediction'])
        _, n_pos_dets, n_neg_dets = calc_frames_removed(result['prediction'])
        _, n_pos_lab, n_neg_lab = calc_frames_removed(label_to_per_frame_list(result['label']))
        perc_frames_removed = 100*(n_neg_dets / len(binary_pred))
        perc_neg_frame_removed = 100*(n_neg_dets / n_neg_lab)
        all_neg_dets += n_neg_dets
        all_neg_lab += n_neg_lab
        all_frames += len(binary_pred)
        #print(f"\n{result['label']['video_id']}: {tfpnr_dict['fpr']:.2f}, {tfpnr_dict['fnr']:.2f}, {tdr:.2f}, {perc_frames_removed:.2f}, {perc_neg_frame_removed:.2f}")
        tp += tfpnr_dict['tp']
        fp += tfpnr_dict['fp']
        fn += tfpnr_dict['fn']
    
    param_set_AP = tp/(tp+fp)
    param_set_AR = tp/(tp+fn)
    param_set_F1 = 2/((1/param_set_AR) + (1/param_set_AP))
    APs.append(param_set_AP)
    ARs.append(param_set_AR)
    F1s.append(param_set_F1)

    if(param_set_AP > maxAP):
        maxAP = param_set_AP
        maxAPParamID = param_set
        maxAPParamIDX = i
    if(param_set_AR > maxAR):
        maxAR = param_set_AR
        maxARParamID = param_set
        maxARParamIDX = i
    if(param_set_F1 > maxF1):
        maxF1 = param_set_F1
        maxF1ParamID = param_set
        maxF1ParamIDX = i
    
    #print(param_set_AP)
    perc_all_frames = 100*(all_neg_dets/all_frames)
    perc_tn_frames = 100*(all_neg_lab/all_frames)
    FRs.append(perc_all_frames)
    # calculate best frame remover
    if(perc_all_frames > maxFramesRemoved):
        maxFR = perc_all_frames
        maxFRParamID = param_set
        maxFRParamIDX = i
    
    
    #print(f"\n mean false negative rate: {mean_fnr:.2f}")
    #print(f"\n% of all frames removed: {perc_all_frames:.2f}")
    #print(f"\n% of all frames that are empty: {perc_tn_frames:.2f}")
print(f"Best Average Precision:\n Param Set: {maxAPParamID} AP: {APs[maxAPParamIDX]:.2f} AR: {ARs[maxAPParamIDX]:.2f} F1: {F1s[maxAPParamIDX]:.2f} FR: {FRs[maxAPParamIDX]:.2f}")
print(f"Best Average Recall:\n Param Set: {maxARParamID} AP: {APs[maxARParamIDX]:.2f} AR: {ARs[maxARParamIDX]:.2f} F1: {F1s[maxARParamIDX]:.2f} FR: {FRs[maxARParamIDX]:.2f}")
print(f"Best F1:\n Param Set: {maxF1ParamID} AP: {APs[maxF1ParamIDX]:.2f} AR: {ARs[maxF1ParamIDX]:.2f} F1: {F1s[maxF1ParamIDX]:.2f} FR: {FRs[maxF1ParamIDX]:.2f}")
print(f"Best Frame Remover:\n Param Set: {maxFRParamID} AP: {APs[maxFRParamIDX]:.2f} AR: {ARs[maxFRParamIDX]:.2f} F1: {F1s[maxFRParamIDX]:.2f} FR: {FRs[maxFRParamIDX]:.2f}")

Best Average Precision:
 Param Set: 200 AP: 0.44 AR: 0.27 F1: 0.34 FR: 88.08
Best Average Recall:
 Param Set: 54 AP: 0.20 AR: 0.88 F1: 0.33 FR: 30.88
Best F1:
 Param Set: 165 AP: 0.35 AR: 0.49 F1: 0.41 FR: 77.49
Best Frame Remover:
 Param Set: 1 AP: 0.26 AR: 0.67 F1: 0.37 FR: 58.15
